In [1]:
import os
import theano
import lasagne
import numpy as np
import theano.tensor as T

Using cuDNN version 5110 on context None
Mapped name None to device cuda0: GRID K520 (0000:00:03.0)


In [2]:
f = open('anna_carenina')

full_text = ''
for line in f:
    full_text += line.decode('utf-8')

In [3]:
from collections import Counter
import re

In [4]:
corpora = full_text.split()

In [5]:
# word_counts = Counter((u" ".join(re.findall(r'[\w]+', full_text, re.U))).split())
word_counts = Counter(full_text.split())
tokens  = ['#UNK#', '#START#', '#END#']
tokens += [k for k, v in word_counts.items() if v >= 5]
n_tokens = len(tokens)
print n_tokens

5908


In [16]:
UNK_ix = tokens.index('#UNK#')
token_to_id = {ch:idx for idx, ch in enumerate(tokens)}
id_to_token = dict(enumerate(tokens))
tokens_ids = np.array([token_to_id.get(ch, UNK_ix) for ch in full_text.split()])

In [17]:
def sample_random_batches(source, n_batches=10, seq_len=20):
    X_batch, y_batch = np.zeros((n_batches, seq_len)), np.zeros(n_batches)
    
    for i in xrange(n_batches):
        pos = np.random.randint(0, source.size - seq_len)
        X_batch[i, :] = source[pos:pos+seq_len]
        y_batch[i] = source[pos+seq_len]

    return X_batch, y_batch

In [18]:
X, y = sample_random_batches(tokens_ids, 4, 5)
print X
print y

[[ 1593.   116.  3928.  2396.  5119.]
 [ 1646.  3250.     0.     0.  3743.]
 [    0.  3799.  1219.  3625.     0.]
 [ 1237.  1646.  1934.  3799.     0.]]
[ 2205.  1187.   313.  5109.]


In [7]:
seq_length = 100 # Как далеко распространяются градиенты
grad_clip = 10 # Максимальный модуль градиента
input_sequence, target_values = T.matrix('input sequence', 'int32'),  T.ivector('target y')

In [8]:
net = lasagne.layers.InputLayer(shape=(None, None), input_var=input_sequence)

net = lasagne.layers.EmbeddingLayer(net, len(tokens), 150)
net = lasagne.layers.RecurrentLayer(net, 500, only_return_final=True)
net = lasagne.layers.DenseLayer(net, len(tokens), nonlinearity=T.nnet.softmax)

In [9]:
# Веса модели
weights = lasagne.layers.get_all_params(net, trainable=True)
print weights

[W, input_to_hidden.W, input_to_hidden.b, hidden_to_hidden.W, W, b]


In [10]:
network_output = lasagne.layers.get_output(net)

In [11]:
loss = lasagne.objectives.categorical_crossentropy(network_output, target_values).mean()
updates = lasagne.updates.adam(loss, weights)

In [12]:
train = theano.function([input_sequence, target_values], loss, updates=updates, allow_input_downcast=True)
compute_cost = theano.function([input_sequence, target_values], loss, allow_input_downcast=True)
probs = theano.function([input_sequence], network_output, allow_input_downcast=True)

In [13]:
def proportional_sample_fun(probs):
    return np.random.choice(len(tokens), p=probs)

In [14]:
# The next function generates text given a phrase of length at least SEQ_LENGTH.

def generate_sample(sample_fun, seed_phrase=None, N=100):
    if seed_phrase is None:
        start = np.random.randint(0,len(corpora)-seq_length)
        seed_phrase = u" ".join(corpora[start:start+seq_length])
        print "Using random seed:", seed_phrase
    while len(seed_phrase) < seq_length:
        seed_phrase = " "+seed_phrase
    if len(seed_phrase) > seq_length:
        seed_phrase = seed_phrase[len(seed_phrase)-seq_length:]
    assert type(seed_phrase) is unicode
           
    sample_ix = []
    x = map(lambda c: token_to_id.get(c,0), seed_phrase)
    x = np.array([x])

    for i in range(N):
        ix = sample_fun(probs(x).ravel())
        if ix != UNK_ix: #token_to_id(u'#UNK#'):
            sample_ix.append(ix)
            x[:,0:seq_length-1] = x[:,1:]
            x[:,seq_length-1] = 0
            x[0,seq_length-1] = ix 

    random_snippet = seed_phrase + u' ' + u' '.join(id_to_token[ix] for ix in sample_ix)    
    print("----\n %s \n----" % random_snippet)

In [ ]:
n_epochs=100 #сколько всего эпох
batches_per_epoch = 1000 # раз в сколько эпох печатать примеры 
batch_size=100 #сколько цепочек обрабатывать за 1 вызов функции обучения

print("Training ...")
for epoch in xrange(n_epochs):
    print "Генерируем текст в пропорциональном режиме"
    generate_sample(proportional_sample_fun,None)

    avg_cost = 0;
    
    for _ in range(batches_per_epoch):
        x,y = sample_random_batches(tokens_ids,batch_size,seq_length)
        avg_cost += train(x, y)
        
    print("Epoch {} average loss = {}".format(epoch, avg_cost / batches_per_epoch))

Training ...
Генерируем текст в пропорциональном режиме
Using random seed: передала от г-жи Шталь сожаление, что она по болезни лишена удовольствия познакомиться с княгиней. Познакомившись с Варенькой, Кити все более и более прельщалась своим другом и с каждым днем находила в ней новые достоинства. Княгиня, услыхав о том, что Варенька хорошо поет, вопросила ее прийти к ним петь вечером. – Кити играет, и у нас есть фортепьяно, нехорошее, правда, но вы нам доставите большое удовольствие, – сказала княгиня с своею притворною улыбкой, которая особенно неприятна была теперь Кити, потому что она заметила, что Вареньке не хотелось петь. Но Варенька, однако, пришла вечером и принесла с собой тетрадь нот. Княгиня пригласила
----
 отелось петь. Но Варенька, однако, пришла вечером и принесла с собой тетрадь нот. Княгиня пригласила землю, спал вышли интересовать остаться засмеялся хотите, куда работать жалею, указывая мужчины скачек qui придет, состояло речи, идет Вареньки хороших впереди зубы, Ми

In [ ]:
seed = u"Каждый человек должен"
sampling_fun = proportional_sample_fun
result_length = 300

generate_sample(sampling_fun,seed,result_length)

In [ ]:
seed = u"В случае неповиновения"
sampling_fun = proportional_sample_fun
result_length = 300

generate_sample(sampling_fun,seed,result_length)